In [1]:
import logging
# set seed
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
import utils as ut
import experiment as exp
from evaluation import *
from sklearn.metrics import mean_squared_error
from torch.utils.tensorboard import SummaryWriter
import torch
import random
#define fixed_hyperparams and create a config gen
from configurations import RandomConfigGen, Configuration
from torch import nn
from deep_net import RandomNet
from experiment import run_experiment
import regex as re
from pathlib import *
from plot import *
from sk_models import setup_pls_models_exh, StandardScaler, PLSRegression, DeepKNN,CustomWrapper
from tqdm.notebook import tqdm, trange

seed = 1
torch.manual_seed(seed)
random.seed(seed + 1)
np.random.seed(seed + 2)
random_state = np.random.RandomState(seed)
import gc
torch.cuda.empty_cache()
gc.collect()

print(f"GPU detected is {torch.cuda.get_device_name(0)}")
plt.style.use('seaborn-darkgrid')

GPU detected is GeForce GTX 970


In [2]:
id_col_db = {'A_C_OF_ALPHA':["sample_id"],
             'A_C_OF_SIWARE':[],
             'A_AL_RT':[],
             'PLN7':["db_id", "sample_id"],
             'mango_684_990': ['Set','Season','Region','Date','Type','Cultivar','Pop','Temp',"FruitID"]
            }

output_col_db= {'A_C_OF_ALPHA':None,
             'A_C_OF_SIWARE':None,
             'A_AL_RT':None,
             'PLN7':None,
             'mango_684_990': ['DM']
            }


In [3]:
#setup input and output formats, load data

file_name = "A_AL_RT.csv"
dataset_name = re.sub(r'\.(?=csv$)[^.]+$', '',file_name)

file_dir= Path("D:/workspace/lazydeep/experiments/2.00")/dataset_name

In [4]:
scores_cv_db = pd.read_csv(file_dir/"scores.csv")
scores_test_db = pd.read_csv(file_dir/"test_scores.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\workspace\\lazydeep\\experiments\\2.00\\A_AL_RT\\scores.csv'

In [ ]:
#order models
deep_set = scores_cv_db[scores_cv_db["predictor"]=="deep"].sort_values("R2")
deep_set["order"] = [i for i in range(0,100)]
deep_ordering = {row["model_num"]:row["order"] for index, row in deep_set.iterrows()}

def order_models(x):
    x = [deep_ordering[i] for i in x]
    return x

In [ ]:
scores_cv_db_deep = scores_cv_db[scores_cv_db["predictor"]=='deep'].sort_values("R2")
scores_cv_db_lr = scores_cv_db[scores_cv_db["predictor"].str.contains('lr')]   #val_eq_list(scores_df["predictor"],'dist')] #np.logical_or(scores_df["predictor"]=="deep",'dist' in scores_df["predictor"])]
scores_cv_db_lwr =scores_cv_db[scores_cv_db["predictor"].str.contains('lwr')] 

In [ ]:
#rank our models on cv_set 

fig, ax = plt.subplots()
set_deep = False
knn_models = scores_cv_db_lwr["predictor"].unique()

ax.plot(order_models(scores_cv_db_deep["model_num"]), scores_cv_db_deep["R2"], label='Deep',c="black")
ax.scatter(x=order_models(scores_cv_db_lr["model_num"].tolist()), y=scores_cv_db_lr["R2"], s=5, label='LR')

for knn_model in knn_models:
    k = int(knn_model.replace("lwr_k=",""))
    if k>=100:
        subset = scores_cv_db[scores_cv_db["predictor"]==knn_model]
        ax.scatter(x=order_models(subset["model_num"].tolist()), y=subset["R2"], s=5, label=f"LWR, k={k}")
#ax.set_ylim(0,scores_db["deep_mean"].max())
ax.set_ylim(0,1)
# plot residuals
ax.legend(loc='upper right',bbox_to_anchor=(1.3, 1))
ax.set_ylabel(r"$R^2$")
ax.set_xlabel("Deep Model Rank")
#ax.set_ylim(0,200)
#ax.set_yscale("symlog")
#ax.set_title("Summary of LWR improvements over Deep Models")
plt.savefig(file_dir/f"prod_cv.png", bbox_inches='tight')

In [ ]:
scores_test_db_deep = scores_test_db[scores_test_db["predictor"]=='deep']
scores_test_db_lr = scores_test_db[scores_test_db["predictor"].str.contains('lr')]   #val_eq_list(scores_df["predictor"],'dist')] #np.logical_or(scores_df["predictor"]=="deep",'dist' in scores_df["predictor"])]
scores_test_db_lwr =scores_test_db[scores_test_db["predictor"].str.contains('lwr')] 

In [ ]:
#rank our models on test_set 

fig, ax = plt.subplots()
set_deep = False
knn_models = scores_test_db_lwr["predictor"].unique()

ax.scatter(x=order_models(scores_test_db_deep["model_num"]),        y= scores_test_db_deep["R2"],s=10, label='Deep',c="black")
ax.scatter(x=order_models(scores_test_db_lr["model_num"].tolist()), y= scores_test_db_lr["R2"] , s=5, label='LR')

for knn_model in knn_models:
    k = int(knn_model.replace("lwr_k=",""))
    if k>=100:
        subset = scores_test_db[scores_test_db["predictor"]==knn_model]
        ax.scatter(x=order_models(subset["model_num"].tolist()), y=subset["R2"], s=5, label=f"LWR k={k}")

#ax.set_ylim(0,scores_db["deep_mean"].max())
ax.set_ylim(0,1)
# plot residuals
ax.legend(loc='upper right',bbox_to_anchor=(1.3, 1))
ax.set_ylabel(r"$R^2$")
ax.set_xlabel("Deep Model Rank")
#ax.set_ylim(0,200)
#ax.set_yscale("symlog")
#ax.set_title("Summary of LWR improvements over Deep Models")
plt.savefig(file_dir/f"prod_test.png", bbox_inches='tight')

In [ ]:
#take our best 30 models 

def filter_db(row,n=30):
    order = deep_ordering[row["model_num"]]
    if order >= (100-n):
        return True
    else:
        return False

inds = scores_cv_db.apply(filter_db,axis=1)
cv_top_30 = scores_cv_db[inds==True]
inds = scores_test_db.apply(filter_db,axis=1)
test_top_30 = scores_test_db[inds==True]

In [ ]:
#rank our models on test_set 
from scipy.stats import linregress
from sklearn.metrics import r2_score


fig, ax = plt.subplots()
set_deep = False
predictors = cv_top_30["predictor"].unique()
for predictor in predictors:
    cv_subset = cv_top_30[cv_top_30["predictor"]==predictor]
    test_subset = test_top_30[test_top_30["predictor"]==predictor]
    x=cv_subset["R2"]
    y=test_subset["R2"]
    R2 = r2_score(x,y)
    lin =linregress(x,y)
    if predictor == "deep":
        ax.scatter(x=x, y=y, s=5,c="black",label=f"Deep")  #R2 = {R2:.4f}")
    elif predictor == "lr":
        ax.scatter(x=x, y=y, s=5, label=f"LR") #R2 = {R2:.4f}")
    else:
        k = int(predictor.replace("lwr_k=",""))
        if k>=100:
            ax.scatter(x=x, y=y, s=5, label=f"LWR k={k}") #R2 = {R2:.4f}")
# plot residuals
ax.legend(loc='upper right' , bbox_to_anchor=(1.3, 1))
ax.set_xlabel(r"CV $R^2$")
ax.set_ylabel(r"Test $R^2$")

#x.set_ylim(0,0.8)
#ax.set_yscale("symlog")
#ax.set_title("Summary of LWR improvements over Deep Models")
plt.savefig(file_dir/f"prod_compare.png", bbox_inches='tight')

In [ ]:
preds_cv_db = pd.read_csv(file_dir/"_predictions.csv")